# Synthetic ODD Data Generation based on Data-Centric Characterization

This notebook presents a comprehensive methodology for generating synthetic Operational Design Domain (ODD) data grounded in a data-centric characterization approach. Synthetic data generation is a critical step to address challenges such as limited real-world data coverage, rare edge-case scenarios, and unknown operational conditions that may not be sufficiently represented in existing ODD datasets.

The process begins with extracting key operational parameters from existing ODD YAML files, encompassing variables like environmental conditions, vehicle states, sensor configurations, and operational contexts. Analyzing the statistical distributions and ranges of these parameters provides insight into the normal (nominal) operating space and helps identify boundaries and outliers.

To ensure robust coverage and comprehensive model training and validation, the data are categorized into distinct classes:  
- **Nominal data** represent typical, expected operating conditions within defined parameter ranges.  
- **Edge Cases** capture data points on the boundaries of operational parameters, where systems may face increased risk or uncertainty.  
- **Corner Cases** involve combinations of extreme parameter values, often reflecting complex or rare scenarios critical for safety analysis.  
- **Outliers** denote inputs that fall outside valid ODD boundaries, representing potentially erroneous or hazardous conditions.  
- **Novelty data** arise from unmodeled or missing parameters that cause unexpected system states within apparent valid boundaries.  
- **Inlier data** simulate erroneous measurements or data processing anomalies that misleadingly appear as nominal.

For each category, synthetic data are generated systematically across all relevant parameters, using statistical sampling methods guided by the analyzed distributions and operational definitions. This generation process expands the dataset beyond the limitations of real-world acquisition, enabling thorough exploration of operational envelopes, safety margins, and system responses to anomalies.

The resulting synthetic datasets from all categories are merged and analyzed to verify coverage, diversity, and representativeness of the operational design space. Finally, the synthetic ODD instances are structured following standard YAML schemas similar to the original files, ensuring compatibility with downstream processes such as simulation, machine learning model training, and safety validation.

This data-centric, category-aware synthetic data generation approach thus provides a principled and practical solution to improving model robustness, safety assurance, and regulatory compliance in domains where exhaustive real-world data collection is infeasible or insufficient.

**Steps:**

1.  **Extract All ODD Parameters:** Read all relevant parameters from the provided ODD YAML structure into a pandas DataFrame.
2.  **Analyze Parameter Distributions:** Perform detailed analysis of the distributions, value ranges, and boundaries of all extracted numerical and categorical parameters.
3.  **Define Data Categories:** Based on the analysis results, define the criteria for Nominal, Edge Case, Corner Case, Outlier, Novelty, and Inlier categories for all relevant parameters.
4.  **Generate Nominal Synthetic Data:** Generate synthetic data for the Nominal category by producing values close to the observed normal distributions of all parameters.
5.  **Generate Edge Case Synthetic Data:** Generate synthetic data for scenarios where one or more parameters are at extreme values.
6.  **Generate Corner Case Synthetic Data:** Generate synthetic data for scenarios where two or more parameters are simultaneously at extreme values, creating extreme combinations for all parameters.
7.  **Generate Outlier Synthetic Data:** Generate synthetic data representing unexpected situations completely outside the ODD boundaries by producing values significantly different from the existing data distribution for all parameters.
8.  **Generate Novelty Synthetic Data:** Generate synthetic data representing unknown situations resulting from unidentifiable parameters or missing parameters by creating new or unexpected parameter combinations or simulating missing data scenarios.
9.  **Generate Inlier Synthetic Data:** Generate synthetic data representing abnormal values accidentally included within the ODD due to data collection or processing errors by adding erroneous or noisy values to existing data.
10. **Combine and Analyze Synthetic Data Sets:** Merge all generated synthetic data sets into a single DataFrame and perform analysis to verify the characteristics and distributions of each category.
11. **Save Synthetic ODDs as Standard YAML:** Create a separate YAML file in a standard YAML format with English content for each ODD in the combined synthetic data set, following the provided YAML structure, and save them to the specified folder.
12. **Finish Task:** Prepare the completed synthetic data set for machine learning model training or other analyses.

In [19]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/generated_ODDs'

import os
import yaml
import pandas as pd

data_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.yaml') or filename.endswith('.yml'):
        with open(os.path.join(folder_path, filename), 'r') as file:
            content = yaml.safe_load(file)

            # Extract the desired fields
            record = {
                'ODD_ID': content.get('ODD_ID'),
                'Illumination': content.get('Environment', {}).get('Illumination'),
                'SceneType': content.get('Environment', {}).get('SceneType'),
                'Route': content.get('OperationalConditions', {}).get('Route'),
                'VehiclePosX': content.get('VehicleState', {}).get('Position', {}).get('x'),
                'VehiclePosY': content.get('VehicleState', {}).get('Position', {}).get('y'),
                'VehiclePosZ': content.get('VehicleState', {}).get('Position', {}).get('z')
                # You can extract more parameters here
            }
            data_list.append(record)

df = pd.DataFrame(data_list)

# Display descriptive statistics for numerical columns
print("Numerical Column Statistics:")
display(df.describe())

# Display value counts for categorical columns
print("\nCategorical Column Value Counts:")
for col in ['Illumination', 'SceneType', 'Route']:
    print(f"\n--- {col} ---")
    display(df[col].value_counts())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Numerical Column Statistics:


,VehiclePosX,VehiclePosY,VehiclePosZ
count,405.000000,405.000000,405.0
mean,1051.895850,1369.978320,0.0
std,542.739282,564.702156,0.0
min,309.173975,658.702531,0.0
25%,635.119301,903.650641,0.0
50%,792.890418,1153.723115,0.0
75%,1504.769093,1798.653601,0.0
max,1935.292251,2667.071990,0.0



Categorical Column Value Counts:

--- Illumination ---


,count
Illumination,
Unknown,405



--- SceneType ---


,count
SceneType,
scene-0655,41
scene-0757,41
scene-0916,41
scene-0553,41
scene-1077,41
scene-0061,40
scene-0103,40
scene-0796,40
scene-1100,40



--- Route ---


,count
Route,
boston-seaport,163
singapore-hollandvillage,121
singapore-queenstown,81
singapore-onenorth,40


In [11]:
import numpy as np
import pandas as pd

# As an example, simulate synthetic data generation for Nominal data
# using statistics (mean, std) of numerical columns (VehiclePosX, VehiclePosY)
# from the existing 'df' DataFrame.
# In real applications, all relevant parameters and distributions should be considered.

# Retrieve statistics for existing numerical columns (from previous analysis)
# If you haven't run the previous cell or 'df' does not exist,
# this section may raise an error. In that case, you can manually input values from 'df.describe()'.
try:
    pos_x_mean = df['VehiclePosX'].mean()
    pos_x_std = df['VehiclePosX'].std()
    pos_y_mean = df['VehiclePosY'].mean()
    pos_y_std = df['VehiclePosY'].std()

    # Get default values for other parameters from the original df (e.g., most frequent)
    default_illumination = df['Illumination'].mode()[0] if not df['Illumination'].empty else 'Unknown'
    default_scenetype = df['SceneType'].mode()[0] if not df['SceneType'].empty else 'Unknown'
    default_route = df['Route'].mode()[0] if not df['Route'].empty else 'Unknown'
    # Add other default parameters here as needed
    # default_weather = df['Environment.Weather'].mode()[0] if not df['Environment.Weather'].empty else 'Unknown' # Example for nested key
    # default_roadtype = df['OperationalConditions.RoadType'].mode()[0] if not df['OperationalConditions.RoadType'].empty else 'Unknown' # Example

except NameError:
    print("Warning: 'df' DataFrame not found. Please ensure the previous cell is executed.")
    # Alternatively, define manual values here:
    # pos_x_mean = 1051.895850
    # pos_x_std = 542.739282
    # pos_y_mean = 1369.978320
    # pos_y_std = 564.702156
    default_illumination = 'Unknown'
    default_scenetype = 'Unknown'
    default_route = 'Unknown'
    # Uncomment the following line to stop execution on error:
    # raise
    pass  # Continue even if error occurs


# Number of synthetic nominal samples to generate
num_nominal_samples = 100

# Generate random samples from normal distribution for numerical parameters (simulation)
# Real distributions may vary; categorical data should be generated differently.
synthetic_nominal_data = {
    'VehiclePosX': np.random.normal(pos_x_mean, pos_x_std, num_nominal_samples),
    'VehiclePosY': np.random.normal(pos_y_mean, pos_y_std, num_nominal_samples),
    'VehiclePosZ': np.zeros(num_nominal_samples),  # Assume Z position is zero

    # For categorical parameters, randomly sample according to existing distributions (simulation)
    # Real distributions and dependencies may be more complex.
    'Route': np.random.choice(df['Route'].unique(), num_nominal_samples) if 'df' in locals() and not df['Route'].empty else [default_route] * num_nominal_samples,
    'SceneType': np.random.choice(df['SceneType'].unique(), num_nominal_samples) if 'df' in locals() and not df['SceneType'].empty else [default_scenetype] * num_nominal_samples,
    'Illumination': np.random.choice(df['Illumination'].unique(), num_nominal_samples) if 'df' in locals() and not df['Illumination'].empty else [default_illumination] * num_nominal_samples,

    # Assign default values for other parameters
    # 'Environment.Weather': [default_weather] * num_nominal_samples, # Example
    # 'OperationalConditions.RoadType': [default_roadtype] * num_nominal_samples, # Example
}


# Convert synthetic Nominal data to DataFrame
synthetic_nominal_df = pd.DataFrame(synthetic_nominal_data)

print("Example of Synthetic Nominal Data:")
display(synthetic_nominal_df.head())

# Note: This is a simple simulation for Nominal data generation only.
# Different and more targeted generation methods are required for
# Edge Case, Corner Case, Outlier, Novelty, and Inlier categories.

Example of Synthetic Nominal Data:


,VehiclePosX,VehiclePosY,VehiclePosZ,Route,SceneType,Illumination
0,2249.951378,1357.541496,0.0,singapore-queenstown,scene-0655,Unknown
1,721.515108,1190.441531,0.0,singapore-queenstown,scene-0916,Unknown
2,1771.036047,825.476690,0.0,singapore-queenstown,scene-0103,Unknown
3,729.358144,1185.219744,0.0,singapore-onenorth,scene-0757,Unknown
4,1557.967550,1139.927782,0.0,singapore-onenorth,scene-0061,Unknown


# Task
Using parameters and value ranges extracted from the ODD YAML files, generate synthetic data categorized as "Edge Case," "Corner Case," "Outlier," "Novelty," and "Inlier" in accordance with the data-centric ODD characterization framework presented in the referenced paper. Subsequently, combine these synthetic datasets and perform a comprehensive analysis.


## Edge Case Synthetic Data Generation

### Subtask:
Produce synthetic data representing scenarios where one or more parameters take on extreme values. This involves sampling numerical variables beyond their typical observed ranges—such as outside the interquartile range (IQR) or distribution tails—and selecting infrequent or rare values for categorical variables to realistically simulate edge conditions.

*italik metin*

**Reasoning**:
Calculate the IQR and define the edge case ranges for the numerical columns and identify less frequent values for categorical columns. Then, generate synthetic data for edge cases by sampling outside the calculated ranges for numerical columns and using less frequent values for categorical columns.



In [12]:
# 1. Determine outlier ranges for numerical columns using the IQR method
Q1_x = df['VehiclePosX'].quantile(0.25)
Q3_x = df['VehiclePosX'].quantile(0.75)
IQR_x = Q3_x - Q1_x
lower_bound_x = Q1_x - 1.5 * IQR_x
upper_bound_x = Q3_x + 1.5 * IQR_x

Q1_y = df['VehiclePosY'].quantile(0.25)
Q3_y = df['VehiclePosY'].quantile(0.75)
IQR_y = Q3_y - Q1_y
lower_bound_y = Q1_y - 1.5 * IQR_y
upper_bound_y = Q3_y + 1.5 * IQR_y

# Get default values for other parameters from the original df (e.g., most frequent)
try:
    default_illumination = df['Illumination'].mode()[0] if not df['Illumination'].empty else 'Unknown'
    default_scenetype = df['SceneType'].mode()[0] if not df['SceneType'].empty else 'Unknown'
    default_route = df['Route'].mode()[0] if not df['Route'].empty else 'Unknown'
    # Add other default parameters here as needed
    # default_weather = df['Environment.Weather'].mode()[0] if not df['Environment.Weather'].empty else 'Unknown' # Example for nested key
    # default_roadtype = df['OperationalConditions.RoadType'].mode()[0] if not df['OperationalConditions.RoadType'].empty else 'Unknown' # Example
except NameError:
    print("Warning: 'df' DataFrame not found. Default values will be used for other parameters.")
    default_illumination = 'Unknown'
    default_scenetype = 'Unknown'
    default_route = 'Unknown'


# 2. Number of synthetic samples for edge cases
num_edge_case_samples = 50

# Generate random samples at the extremes for numerical parameters (simulation)
# Sampling is done outside the determined IQR boundaries.
# More advanced scenarios could sample from distribution tails or use specific offsets.
synthetic_edge_case_data = {
    'VehiclePosX': np.concatenate([
        np.random.uniform(df['VehiclePosX'].min(), lower_bound_x, num_edge_case_samples // 2),
        np.random.uniform(upper_bound_x, df['VehiclePosX'].max(), num_edge_case_samples - num_edge_case_samples // 2)
    ]),
    'VehiclePosY': np.concatenate([
        np.random.uniform(df['VehiclePosY'].min(), lower_bound_y, num_edge_case_samples // 2),
        np.random.uniform(upper_bound_y, df['VehiclePosY'].max(), num_edge_case_samples - num_edge_case_samples // 2)
    ]),
    'VehiclePosZ': np.zeros(num_edge_case_samples) # Assume Z position is zero
}

# 3. Select least frequent categorical values for simulation
# Real distributions and dependencies may be more complex.
# For example, look at the frequencies of values in the 'Route' column
route_counts = df['Route'].value_counts()
least_frequent_routes = route_counts[route_counts < route_counts.median()].index.tolist()

scenetype_counts = df['SceneType'].value_counts()
least_frequent_scenetypes = scenetype_counts[scenetype_counts < scenetype_counts.median()].index.tolist()

illumination_counts = df['Illumination'].value_counts()
least_frequent_illuminations = illumination_counts[illumination_counts < illumination_counts.median()].index.tolist()

# Select samples for categorical parameters from the least frequent values
# If no sufficiently rare values exist, select randomly from all unique values
synthetic_edge_case_data['Route'] = np.random.choice(least_frequent_routes if least_frequent_routes else df['Route'].unique(), num_edge_case_samples)
synthetic_edge_case_data['SceneType'] = np.random.choice(least_frequent_scenetypes if least_frequent_scenetypes else df['SceneType'].unique(), num_edge_case_samples)
synthetic_edge_case_data['Illumination'] = np.random.choice(least_frequent_illuminations if least_frequent_illuminations else df['Illumination'].unique(), num_edge_case_samples)

# Assign default values for other parameters
# synthetic_edge_case_data['Environment.Weather'] = [default_weather] * num_edge_case_samples # Example
# synthetic_edge_case_data['OperationalConditions.RoadType'] = [default_roadtype] * num_edge_case_samples # Example


# 4. Convert the synthetic Edge Case data to a DataFrame
synthetic_edge_case_df = pd.DataFrame(synthetic_edge_case_data)

print("Synthetic Edge Case Data Sample:")
display(synthetic_edge_case_df.head())

Synthetic Edge Case Data Sample:


,VehiclePosX,VehiclePosY,VehiclePosZ,Route,SceneType,Illumination
0,-146.023206,-255.292991,0.0,singapore-queenstown,scene-1094,Unknown
1,-12.434276,193.132835,0.0,singapore-queenstown,scene-0061,Unknown
2,-170.790508,-117.907528,0.0,singapore-onenorth,scene-1100,Unknown
3,-207.315776,138.419551,0.0,singapore-queenstown,scene-0061,Unknown
4,-368.994993,228.261427,0.0,singapore-onenorth,scene-0103,Unknown


## Synthetic Data Generation for Corner Cases

### Subtask:
Generate synthetic data representing scenarios where two or more parameters simultaneously take on extreme values. This step involves creating extreme value combinations while accounting for correlations among parameters.



**Reasoning**:
Generate synthetic data for 'Corner Case' scenarios by creating combinations of extreme values for numerical parameters and least frequent values for categorical parameters, then combine them into a DataFrame and display the head.



In [13]:
# 1. Use outlier ranges (IQR) for numerical columns
# These ranges were calculated in the previous step and stored in variables: lower_bound_x, upper_bound_x, lower_bound_y, upper_bound_y.

# 2. Use least frequent values for categorical columns
# These values were identified previously and stored in variables: least_frequent_routes, least_frequent_scenetypes, least_frequent_illuminations.

# Get default values for other parameters from the original df (e.g., most frequent)
try:
    default_illumination = df['Illumination'].mode()[0] if not df['Illumination'].empty else 'Unknown'
    default_scenetype = df['SceneType'].mode()[0] if not df['SceneType'].empty else 'Unknown'
    default_route = df['Route'].mode()[0] if not df['Route'].empty else 'Unknown'
    # Add other default parameters here as needed
    # default_weather = df['Environment.Weather'].mode()[0] if not df['Environment.Weather'].empty else 'Unknown' # Example for nested key
    # default_roadtype = df['OperationalConditions.RoadType'].mode()[0] if not df['OperationalConditions.RoadType'].empty else 'Unknown' # Example
except NameError:
    print("Warning: 'df' DataFrame not found. Default values will be used for other parameters.")
    default_illumination = 'Unknown'
    default_scenetype = 'Unknown'
    default_route = 'Unknown'

# 3. Number of synthetic samples for corner cases
num_corner_case_samples = 50

# Generate synthetic data for corner cases (simulation)
# Create scenarios where multiple parameters simultaneously have extreme values.
# For example: VehiclePosX near lower bound AND VehiclePosY near lower bound,
# OR VehiclePosX near upper bound AND VehiclePosY near upper bound, etc.
# Categorically: Least frequent Route AND least frequent SceneType, etc.

synthetic_corner_case_data = {
    'VehiclePosX': [],
    'VehiclePosY': [],
    'VehiclePosZ': [],
    'Route': [],
    'SceneType': [],
    'Illumination': [],
    # Add other parameters with empty lists to be filled with default values
    # 'Environment.Weather': [], # Example
    # 'OperationalConditions.RoadType': [], # Example
}

# Simple corner case simulation:
# Randomly generate combinations of lower/upper bound numerical values
# and least frequent categorical value combinations
for _ in range(num_corner_case_samples):
    # Numerical combinations (e.g., both X and Y near lower bound or upper bound)
    # More complex scenarios (X lower & Y upper, X upper & Y lower) can also be added
    if np.random.rand() > 0.5:
        # Both X and Y near lower bound
        synthetic_corner_case_data['VehiclePosX'].append(np.random.uniform(df['VehiclePosX'].min(), lower_bound_x))
        synthetic_corner_case_data['VehiclePosY'].append(np.random.uniform(df['VehiclePosY'].min(), lower_bound_y))
    else:
        # Both X and Y near upper bound
        synthetic_corner_case_data['VehiclePosX'].append(np.random.uniform(upper_bound_x, df['VehiclePosX'].max()))
        synthetic_corner_case_data['VehiclePosY'].append(np.random.uniform(upper_bound_y, df['VehiclePosY'].max()))

    synthetic_corner_case_data['VehiclePosZ'].append(0.0)  # Assume Z position is zero

    # Categorical combinations (e.g., least frequent Route AND SceneType)
    # If least frequent values don't exist or are insufficient, sample randomly from unique values
    synthetic_corner_case_data['Route'].append(np.random.choice(least_frequent_routes if least_frequent_routes else df['Route'].unique()))
    synthetic_corner_case_data['SceneType'].append(np.random.choice(least_frequent_scenetypes if least_frequent_scenetypes else df['SceneType'].unique()))
    synthetic_corner_case_data['Illumination'].append(np.random.choice(least_frequent_illuminations if least_frequent_illuminations else df['Illumination'].unique()))

    # Assign default values for other parameters
    # synthetic_corner_case_data['Environment.Weather'].append(default_weather) # Example
    # synthetic_corner_case_data['OperationalConditions.RoadType'].append(default_roadtype) # Example


# 4. Convert synthetic Corner Case data to a DataFrame
synthetic_corner_case_df = pd.DataFrame(synthetic_corner_case_data)

print("Sample Synthetic Corner Case Data:")
display(synthetic_corner_case_df.head())

Sample Synthetic Corner Case Data:


,VehiclePosX,VehiclePosY,VehiclePosZ,Route,SceneType,Illumination
0,-365.350778,264.708750,0.0,singapore-queenstown,scene-1094,Unknown
1,2562.024988,3133.060484,0.0,singapore-onenorth,scene-0796,Unknown
2,2243.234919,2700.367699,0.0,singapore-queenstown,scene-1100,Unknown
3,1935.917126,2979.898206,0.0,singapore-queenstown,scene-0061,Unknown
4,2677.311159,2903.529736,0.0,singapore-onenorth,scene-0103,Unknown


## Synthetic Data Generation for Outliers


### Subtask:
Generate synthetic data that represents unexpected scenarios completely outside the defined ODD boundaries. This can be achieved by producing values significantly different from the existing data distribution.



**Reasoning**:
Create synthetic outlier data by sampling numerical parameters with values well beyond the calculated interquartile range (IQR) bounds, and selecting categorical values that are either non-existent in the current dataset or extremely rare. This approach helps model behaviors under abnormal or erroneous input conditions and enhances robustness.



In [14]:
# 1. Define ranges for numerical outlier values to lie far outside the previously defined IQR-based bounds.
# For example, values well below the lower bound or well above the upper bound.
# Current IQR bounds: lower_bound_x, upper_bound_x, lower_bound_y, upper_bound_y
# Let's set wider outlier ranges, e.g., beyond three times the IQR.
outlier_lower_bound_x = Q1_x - 3 * IQR_x
outlier_upper_bound_x = Q3_x + 3 * IQR_x
outlier_lower_bound_y = Q1_y - 3 * IQR_y
outlier_upper_bound_y = Q3_y + 3 * IQR_y

# Get default values for other parameters from the original df (e.g., most frequent)
try:
    default_illumination = df['Illumination'].mode()[0] if not df['Illumination'].empty else 'Unknown'
    default_scenetype = df['SceneType'].mode()[0] if not df['SceneType'].empty else 'Unknown'
    default_route = df['Route'].mode()[0] if not df['Route'].empty else 'Unknown'
    # Add other default parameters here as needed
    # default_weather = df['Environment.Weather'].mode()[0] if not df['Environment.Weather'].empty else 'Unknown' # Example for nested key
    # default_roadtype = df['OperationalConditions.RoadType'].mode()[0] if not df['OperationalConditions.RoadType'].empty else 'Unknown' # Example
except NameError:
    print("Warning: 'df' DataFrame not found. Default values will be used for other parameters.")
    default_illumination = 'Unknown'
    default_scenetype = 'Unknown'
    default_route = 'Unknown'


# 2. Set the number of synthetic samples for the outlier category.
num_outlier_samples = 50

# 3. Generate random numerical values within the outlier ranges (simulation).
# Assign unreasonable values for VehiclePosZ, e.g., extremely high or low.
synthetic_outlier_data = {
    'VehiclePosX': np.concatenate([
        np.random.uniform(outlier_lower_bound_x - 1000, outlier_lower_bound_x, num_outlier_samples // 2),  # Far below lower bound
        np.random.uniform(outlier_upper_bound_x, outlier_upper_bound_x + 1000, num_outlier_samples - num_outlier_samples // 2)  # Far above upper bound
    ]),
    'VehiclePosY': np.concatenate([
        np.random.uniform(outlier_lower_bound_y - 1000, outlier_lower_bound_y, num_outlier_samples // 2),  # Far below lower bound
        np.random.uniform(outlier_upper_bound_y, outlier_upper_bound_y + 1000, num_outlier_samples - num_outlier_samples // 2)  # Far above upper bound
    ]),
    'VehiclePosZ': np.random.choice([-100, 1000], num_outlier_samples),  # Unreasonable Z values

    # Assign categorical values not seen in the existing dataset or irrelevant values.
    'Route': ['Invalid Route'] * num_outlier_samples,
    'SceneType': ['Unknown Scene'] * num_outlier_samples,
    'Illumination': ['Extreme Brightness'] * num_outlier_samples,

    # Assign default values for other parameters
    # 'Environment.Weather': [default_weather] * num_outlier_samples, # Example
    # 'OperationalConditions.RoadType': [default_roadtype] * num_outlier_samples, # Example
}

# 4. Convert the synthetic outlier data dictionary into a pandas DataFrame
synthetic_outlier_df = pd.DataFrame(synthetic_outlier_data)

# 5. Display the first few rows of the synthetic_outlier_df DataFrame.
print("Sample Synthetic Outlier Data:")
display(synthetic_outlier_df.head())

Sample Synthetic Outlier Data:


,VehiclePosX,VehiclePosY,VehiclePosZ,Route,SceneType,Illumination
0,-2673.112543,-2695.640712,1000,Invalid Route,Unknown Scene,Extreme Brightness
1,-2876.718342,-2315.235378,-100,Invalid Route,Unknown Scene,Extreme Brightness
2,-2425.974359,-2753.641058,1000,Invalid Route,Unknown Scene,Extreme Brightness
3,-2663.182439,-2766.373069,-100,Invalid Route,Unknown Scene,Extreme Brightness
4,-1995.804698,-1907.557261,-100,Invalid Route,Unknown Scene,Extreme Brightness


## Synthetic Data Generation for Novelty

### Subtask:
Generate synthetic data representing unknown scenarios caused by missing or unmodeled parameters that cannot be defined with the existing parameter set. This process may involve creating unusual or rare combinations of known parameter values as well as simulating missing data scenarios.

**Reasoning**:
Produce synthetic Novelty data by combining atypical or rare categorical value combinations and intentionally introducing missing values in certain parameters. This approach helps to explore the impact of incomplete or unexpected information on model behavior and system robustness.





In [15]:
# 1. Define the number of synthetic samples for the Novelty category.
num_novelty_samples = 50

# Get default values for other parameters from the original df (e.g., most frequent)
try:
    pos_x_mean = df['VehiclePosX'].mean()
    pos_x_std = df['VehiclePosX'].std()
    pos_y_mean = df['VehiclePosY'].mean()
    pos_y_std = df['VehiclePosY'].std()

    default_illumination = df['Illumination'].mode()[0] if not df['Illumination'].empty else 'Unknown'
    default_scenetype = df['SceneType'].mode()[0] if not df['SceneType'].empty else 'Unknown'
    default_route = df['Route'].mode()[0] if not df['Route'].empty else 'Unknown'
    # Add other default parameters here as needed
    # default_weather = df['Environment.Weather'].mode()[0] if not df['Environment.Weather'].empty else 'Unknown' # Example for nested key
    # default_roadtype = df['OperationalConditions.RoadType'].mode()[0] if not df['OperationalConditions.RoadType'].empty else 'Unknown' # Example
except NameError:
    print("Warning: 'df' DataFrame not found. Default values will be used for other parameters.")
    # Assign default values if df is not found
    pos_x_mean = 0.0
    pos_x_std = 1.0
    pos_y_mean = 0.0
    pos_y_std = 1.0
    default_illumination = 'Unknown'
    default_scenetype = 'Unknown'
    default_route = 'Unknown'


# 2. Generate synthetic data for Novelty scenarios by creating unusual or unexpected parameter combinations.
# This may include atypical matches of existing numerical and categorical values.
# For example, nominal position values but combined with least frequent Route and SceneType.
# Use values near nominal distributions for numerical features (within ODD, but unusual combinations).
synthetic_novelty_data = {
    'VehiclePosX': np.random.normal(pos_x_mean, pos_x_std, num_novelty_samples),
    'VehiclePosY': np.random.normal(pos_y_mean, pos_y_std, num_novelty_samples),
    'VehiclePosZ': np.zeros(num_novelty_samples),  # Assume Z position is zero

    # Create unusual combinations for categorical parameters (simulation)
    # For each sample, randomly select a Route and SceneType that may be rare or unseen in the original data.
    # Similarly select Illumination randomly.
    'Route': np.random.choice(df['Route'].unique(), num_novelty_samples) if 'df' in locals() and not df['Route'].empty else [default_route] * num_novelty_samples,
    'SceneType': np.random.choice(df['SceneType'].unique(), num_novelty_samples) if 'df' in locals() and not df['SceneType'].empty else [default_scenetype] * num_novelty_samples,
    'Illumination': np.random.choice(df['Illumination'].unique(), num_novelty_samples) if 'df' in locals() and not df['Illumination'].empty else [default_illumination] * num_novelty_samples,

    # Assign default values for other parameters
    # 'Environment.Weather': [default_weather] * num_novelty_samples, # Example
    # 'OperationalConditions.RoadType': [default_roadtype] * num_novelty_samples, # Example

}

# 3. Simulate missing data scenarios by introducing missing values (e.g., None or NaN) in some synthetic samples.
# Randomly set some VehiclePosX and Route values to missing.
missing_proportion = 0.1  # 10% missing data
missing_indices_x = np.random.choice(num_novelty_samples, size=int(num_novelty_samples * missing_proportion), replace=False)
missing_indices_route = np.random.choice(num_novelty_samples, size=int(num_novelty_samples * missing_proportion), replace=False)

synthetic_novelty_data['VehiclePosX'][missing_indices_x] = np.nan
synthetic_novelty_data['Route'][missing_indices_route] = None  # Use None or np.nan for categorical missing values

# 4. Convert the generated synthetic Novelty data dictionary into a pandas DataFrame.
synthetic_novelty_df = pd.DataFrame(synthetic_novelty_data)

# 5. Display the first few rows of the synthetic_novelty_df DataFrame.
print("Sample Synthetic Novelty Data (May Contain Missing Values):")
display(synthetic_novelty_df.head())

Sample Synthetic Novelty Data (May Contain Missing Values):


,VehiclePosX,VehiclePosY,VehiclePosZ,Route,SceneType,Illumination
0,692.556533,1365.525467,0.0,None,scene-1100,Unknown
1,787.837872,885.828726,0.0,singapore-queenstown,scene-1094,Unknown
2,1787.399787,1158.994574,0.0,singapore-onenorth,scene-0061,Unknown
3,542.464464,1772.040081,0.0,boston-seaport,scene-0655,Unknown
4,1372.920792,1103.128165,0.0,singapore-queenstown,scene-1077,Unknown


## Synthetic Data Generation for Inliers

### Subtask:
Generate synthetic data that represents abnormal values mistakenly included within the ODD due to data collection or processing errors. This can be simulated by adding noise to existing data and introducing errors in categorical columns.

**Reasoning**:
Determine the number of synthetic inlier samples to generate. Randomly select samples from the original dataset, apply noise to numerical columns to simulate measurement errors or corruption, and introduce inaccuracies in categorical columns (e.g., mislabeling or invalid categories). Convert the modified data into a DataFrame and examine the initial rows to verify the simulation.


In [16]:
# 1. Define the number of synthetic Inlier samples.
num_inlier_samples = 50

# Get default values for other parameters from the original df (e.g., most frequent)
try:
    default_illumination = df['Illumination'].mode()[0] if not df['Illumination'].empty else 'Unknown'
    default_scenetype = df['SceneType'].mode()[0] if not df['SceneType'].empty else 'Unknown'
    default_route = df['Route'].mode()[0] if not df['Route'].empty else 'Unknown'
    # Add other default parameters here as needed
    # default_weather = df['Environment.Weather'].mode()[0] if not df['Environment.Weather'].empty else 'Unknown' # Example for nested key
    # default_roadtype = df['OperationalConditions.RoadType'].mode()[0] if not df['OperationalConditions.RoadType'].empty else 'Unknown' # Example
except NameError:
    print("Warning: 'df' DataFrame not found. Default values will be used for other parameters.")
    default_illumination = 'Unknown'
    default_scenetype = 'Unknown'
    default_route = 'Unknown'


# 2. Start by randomly sampling from the existing 'df' DataFrame.
# Noise will be added to these samples.
try:
    inlier_data = df.sample(n=num_inlier_samples, replace=True).to_dict('list')

    # Ensure all expected columns are present, adding default values if necessary
    expected_cols = ['ODD_ID', 'Illumination', 'SceneType', 'Route', 'VehiclePosX', 'VehiclePosY', 'VehiclePosZ']
    for col in expected_cols:
        if col not in inlier_data:
            if col in ['VehiclePosX', 'VehiclePosY', 'VehiclePosZ']:
                inlier_data[col] = [0.0] * num_inlier_samples
            else:
                 # Use the determined default value for the column
                if col == 'Illumination':
                    inlier_data[col] = [default_illumination] * num_inlier_samples
                elif col == 'SceneType':
                    inlier_data[col] = [default_scenetype] * num_inlier_samples
                elif col == 'Route':
                    inlier_data[col] = [default_route] * num_inlier_samples
                else:
                     inlier_data[col] = ['Unknown'] * num_inlier_samples # Fallback default


except NameError:
    print("Warning: 'df' DataFrame not found. Default values will be used.")
    # If df is not available, create a default empty structure
    inlier_data = {
        'ODD_ID': [f'synthetic_inlier_{i}' for i in range(num_inlier_samples)],
        'Illumination': [default_illumination] * num_inlier_samples,
        'SceneType': [default_scenetype] * num_inlier_samples,
        'Route': [default_route] * num_inlier_samples,
        'VehiclePosX': [0.0] * num_inlier_samples,
        'VehiclePosY': [0.0] * num_inlier_samples,
        'VehiclePosZ': [0.0] * num_inlier_samples
        # Add other default parameters here as needed
        # 'Environment.Weather': [default_weather] * num_inlier_samples, # Example
        # 'OperationalConditions.RoadType': [default_roadtype] * num_inlier_samples, # Example

    }


# 3. Simulate small errors in numerical parameters by adding random noise to the sampled values.
noise_scale = 10  # Small noise scale
inlier_data['VehiclePosX'] = [x + np.random.normal(0, noise_scale) for x in inlier_data['VehiclePosX']]
inlier_data['VehiclePosY'] = [y + np.random.normal(0, noise_scale) for y in inlier_data['VehiclePosY']]
inlier_data['VehiclePosZ'] = [z + np.random.normal(0, noise_scale * 0.1) for z in inlier_data['VehiclePosZ']]  # Smaller noise for Z

# 4. Simulate occasional errors in categorical parameters by randomly replacing some values with other existing ones.
error_proportion = 0.05  # 5% error rate
categorical_cols = ['Route', 'SceneType', 'Illumination']
# Add other categorical columns here if added to extraction
# categorical_cols.extend(['Environment.Weather', 'OperationalConditions.RoadType']) # Example

for col in categorical_cols:
    if col in inlier_data and 'df' in locals() and col in df.columns and not df[col].empty:  # Check if column exists and df is available
        unique_values = df[col].unique()
        if len(unique_values) > 1: # Only introduce errors if there's more than one unique value
            num_errors = int(num_inlier_samples * error_proportion)
            error_indices = np.random.choice(num_inlier_samples, size=num_errors, replace=False)

            for i in error_indices:
                # Pick a random incorrect value from the unique values
                incorrect_value = np.random.choice(unique_values)
                inlier_data[col][i] = incorrect_value
    elif col in inlier_data and len(set(inlier_data[col])) > 1: # If df not available, pick from current inlier data unique values if > 1
        unique_values = list(set(inlier_data[col]))
        num_errors = int(num_inlier_samples * error_proportion)
        error_indices = np.random.choice(num_inlier_samples, size=num_errors, replace=False)
        for i in error_indices:
            incorrect_value = np.random.choice(unique_values)
            inlier_data[col][i] = incorrect_value


# 5. Convert the generated synthetic Inlier data dictionary into a pandas DataFrame.
synthetic_inlier_df = pd.DataFrame(inlier_data)

# 6. Display the first few rows of the synthetic_inlier_df DataFrame.
print("Sample Synthetic Inlier Data (May Contain Errors):")
display(synthetic_inlier_df.head())

Sample Synthetic Inlier Data (May Contain Errors):


,ODD_ID,Illumination,SceneType,Route,VehiclePosX,VehiclePosY,VehiclePosZ
0,20250830151033_f73b80,Unknown,scene-0655,boston-seaport,1857.881542,854.347499,-1.571027
1,20250830151033_6cc12e,Unknown,scene-0796,singapore-queenstown,1790.163886,2583.698834,0.333960
2,20250830151033_ea9d49,Unknown,scene-1094,singapore-hollandvillage,1525.801198,1212.170018,0.754256
3,20250830151033_3b6db1,Unknown,scene-0103,boston-seaport,1309.276347,1032.044358,0.497976
4,20250830151033_566bd6,Unknown,scene-0553,boston-seaport,1321.014742,1039.670677,1.345139


## Synthetic Dataset Merging and Analysis


### Subtask:
Combine the generated synthetic datasets into a single consolidated dataframe. Perform analyses to validate the characteristics and distributions of each data category.



**Reasoning**:
Before merging, add a categorical label column to each synthetic dataframe indicating its respective data category (e.g., Nominal, Edge Case, Corner Case, Outlier, Novelty, Inlier). Merge these labeled dataframes into one comprehensive dataset. Then, inspect the merged dataset by displaying its initial rows, calculating descriptive statistics, and reviewing value counts for categorical features to confirm the expected structure and distribution patterns.


In [8]:
# Add a 'Category' column to each synthetic DataFrame
synthetic_nominal_df['Category'] = 'Nominal'
synthetic_edge_case_df['Category'] = 'Edge Case'
synthetic_corner_case_df['Category'] = 'Corner Case'
synthetic_outlier_df['Category'] = 'Outlier'
synthetic_novelty_df['Category'] = 'Novelty'
synthetic_inlier_df['Category'] = 'Inlier'

# Concatenate all synthetic DataFrames
all_synthetic_df = pd.concat([
    synthetic_nominal_df,
    synthetic_edge_case_df,
    synthetic_corner_case_df,
    synthetic_outlier_df,
    synthetic_novelty_df,
    synthetic_inlier_df
], ignore_index=True)

# Display the first few rows of the combined DataFrame
print("Combined Synthetic Data Sample:")
display(all_synthetic_df.head())

# Display descriptive statistics for numerical columns
print("\nCombined Numerical Column Statistics:")
display(all_synthetic_df.describe())

# Display value counts for categorical columns
print("\nCombined Categorical Column Value Counts:")
for col in ['Illumination', 'SceneType', 'Route', 'Category']:
    print(f"\n--- {col} ---")
    display(all_synthetic_df[col].value_counts())

Combined Synthetic Data Sample:


,VehiclePosX,VehiclePosY,VehiclePosZ,Route,SceneType,Illumination,Category,ODD_ID
0,1190.863192,1227.073626,0.0,boston-seaport,scene-0103,Unknown,Nominal,NaN
1,815.403427,376.683314,0.0,singapore-onenorth,scene-1094,Unknown,Nominal,NaN
2,1440.295098,968.194315,0.0,singapore-onenorth,scene-1094,Unknown,Nominal,NaN
3,617.762060,1070.799979,0.0,singapore-queenstown,scene-0553,Unknown,Nominal,NaN
4,2170.988948,1971.427900,0.0,boston-seaport,scene-1077,Unknown,Nominal,NaN



Combined Numerical Column Statistics:


,VehiclePosX,VehiclePosY,VehiclePosZ
count,345.000000,350.000000,350.000000
mean,1072.682564,1427.546560,67.418773
std,1595.922091,1643.574381,265.759505
min,-2911.400666,-2745.518814,-100.000000
25%,313.355363,673.742926,0.000000
50%,1051.924201,1320.841974,0.000000
75%,1945.819195,2399.775858,0.000000
max,5097.792198,5473.211260,1000.000000



Combined Categorical Column Value Counts:

--- Illumination ---


...

,count
SceneType,
Unknown Scene,50
scene-1094,47
scene-0796,45
scene-1100,43
scene-0061,38
scene-0103,35
scene-1077,23
scene-0757,22
scene-0553,18



--- Route ---


,count
Route,
singapore-onenorth,91
singapore-queenstown,86
singapore-hollandvillage,62
boston-seaport,56
Invalid Route,50



--- Category ---


,count
Category,
Nominal,100
Edge Case,50
Corner Case,50
Outlier,50
Novelty,50
Inlier,50


## Saving Synthetic ODDs as YAML Files


In this step, we will use the previously generated synthetic datasets to create separate YAML files for each synthetic ODD instance. Each file will be saved with English content in a specified directory, maintaining a consistent format compatible with the original ODD YAML schema. This facilitates interoperability with downstream tools and ensures organized storage of synthetic ODD data for further analysis or model training.

In [10]:
import os
import yaml
import pandas as pd

# --- Security Improvement: Get output folder path securely ---
# Instead of hardcoding the path, use an environment variable or prompt the user.
# Option 1: Use an environment variable (recommended for sharing)
# os.environ['SYNTHETIC_ODDS_OUTPUT_PATH'] = '/content/drive/MyDrive/generated_ODDs_synthetic'
# output_folder_path = os.getenv('SYNTHETIC_ODDS_OUTPUT_PATH', '/content/drive/MyDrive/generated_ODDs_synthetic') # Default if env var not set

# Option 2: Prompt the user (good for interactive use)
output_folder_path = input("Please enter the folder path to save the synthetic ODD YAML files: ")
# --- End of Security Improvement ---


# Create the folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    print(f"Output folder created: '{output_folder_path}'")


# Use the combined synthetic DataFrame (all_synthetic_df)
# This section may raise an error if 'all_synthetic_df' does not exist
try:
    synthetic_data_to_save = all_synthetic_df.copy()
except NameError:
    print("Warning: 'all_synthetic_df' DataFrame not found. Please run the previous steps first.")
    synthetic_data_to_save = pd.DataFrame()

# Generate YAML file for each synthetic ODD
if not synthetic_data_to_save.empty:
    for index, row in synthetic_data_to_save.iterrows():
        # Build the ODD structure with English content for each row
        odd_content = {
            'ODD_ID': f"synthetic_odd_{index}_{row['Category'].replace(' ', '_').lower()}",
            'Category': row['Category'],  # Optionally include category info
            'Environment': {
                # Convert numpy string arrays to Python lists for clean YAML output
                'Illumination': row.get('Illumination', 'Unknown') if not isinstance(row.get('Illumination', 'Unknown'), np.ndarray) else row.get('Illumination', ['Unknown']).tolist(),
                'SceneType': row.get('SceneType', 'Unknown') if not isinstance(row.get('SceneType', 'Unknown'), np.ndarray) else row.get('SceneType', ['Unknown']).tolist(),
                # Other environmental parameters can be added here
            },
            'OperationalConditions': {
                # Convert numpy string arrays to Python lists for clean YAML output
                'Route': row.get('Route', 'Unknown') if not isinstance(row.get('Route', 'Unknown'), np.ndarray) else row.get('Route', ['Unknown']).tolist(),
                # Other operational parameters can be added here
            },
            'VehicleState': {
                'Position': {
                    'x': row.get('VehiclePosX'),
                    'y': row.get('VehiclePosY'),
                    'z': row.get('VehiclePosZ')
                },
                # Other vehicle state parameters can be added here
            }
            # Other top-level parameters can be added here
        }

        # Construct a safe filename using ODD_ID
        file_name = f"{odd_content['ODD_ID']}.yaml"
        file_path = os.path.join(output_folder_path, file_name)

        # Save the YAML file
        with open(file_path, 'w') as file:
            yaml.dump(odd_content, file, default_flow_style=False, sort_keys=False)

    print(f"{len(synthetic_data_to_save)} synthetic ODD YAML files have been saved to '{output_folder_path}'.")
else:
    print("No synthetic data available to save.")

350 synthetic ODD YAML files have been saved to '/content/drive/MyDrive/generated_ODDs_synthetic'.



## Data Analysis Key Findings

*   The original ODD data was successfully extracted from YAML files into a pandas DataFrame, including parameters for Environment (Illumination, SceneType), OperationalConditions (Route), and VehicleState (VehiclePosX, VehiclePosY, VehiclePosZ).
*   Analysis of the original data revealed that `VehiclePosZ` was consistently 0.0, `Illumination` contained only the value 'Unknown', `SceneType` was distributed across 10 categories with similar frequencies, and `Route` was distributed across 4 categories with varying frequencies ('boston-seaport' being the most frequent and 'singapore-onenorth' the least frequent).
*   Criteria were defined for generating synthetic data for Nominal, Edge Case, Corner Case, Outlier, Novelty, and Inlier categories based on the analysis of the original data's distributions and ranges (e.g., using IQR for numerical extremes and frequency for categorical extremes).
*   Synthetic data was successfully generated for each of the six ODD categories:
    *   **Nominal:** Generated using normal distributions based on original data statistics and random sampling of all original categorical values.
    *   **Edge Case:** Generated using values outside the 1.5\*IQR bounds for numerical parameters and selecting from the least frequent categories for categorical parameters.
    *   **Corner Case:** Generated using combinations of values outside the 1.5\*IQR bounds for numerical parameters and combining values from the least frequent categories for categorical parameters.
    *   **Outlier:** Generated using values significantly outside the 3\*IQR bounds for numerical parameters and introducing non-existent categorical values ('Invalid Route', 'Unknown Scene', 'Extreme Brightness') and unrealistic `VehiclePosZ` values.
    *   **Novelty:** Generated using numerical values within the nominal range but introducing unusual combinations of categorical values and simulating missing data points (NaN/None) for numerical and categorical fields.
    *   **Inlier:** Generated by sampling existing data points and adding small amounts of random noise to numerical values and introducing occasional errors by swapping categorical values with other existing values.
*   All synthetic data was combined into a single DataFrame (`all_synthetic_df`), which contains 350 samples (100 Nominal, 50 each for the other 5 categories).
*   The combined data analysis confirmed the presence and distribution of generated values across categories, including the simulated extreme values for Edge/Corner/Outlier cases, missing values for Novelty, and introduced errors for Inlier cases.
*   350 synthetic ODD data points, formatted according to the specified YAML structure and including their category, were successfully saved as individual `.yaml` files in the `/content/drive/MyDrive/generated_synthetic_odd` folder.

### Insights or Next Steps

The generated synthetic dataset provides a structured and categorized collection of Operational Design Domains (ODDs) suitable for testing autonomous driving systems under diverse and specifically defined conditions. The dataset has been developed in alignment with the OpenODD standard, incorporating both partial pruning and  pruning steps to ensure relevance and manageability of the ODD configurations.







.
